In [1]:
import numpy as np
import pandas as pd
import glob
from datetime import datetime
import xgboost as xgb
import lightgbm as lgb
pd.options.display.max_columns = None

from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/biens_2015-19.csv')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765694 entries, 0 to 765693
Data columns (total 10 columns):
valeur_fonciere              765694 non-null float64
code_postal                  765694 non-null float64
commune                      765694 non-null object
code_departement             765694 non-null int64
type_local                   765694 non-null object
surface_reelle_bati          765694 non-null float64
nombre_pieces_principales    765694 non-null int64
surface_terrain              765694 non-null float64
year                         765694 non-null int64
month                        765694 non-null int64
dtypes: float64(4), int64(4), object(2)
memory usage: 58.4+ MB


## Encode categorical data

In [4]:
categories = ['code_departement', 'code_postal', 'type_local'] #, 'month', 'year'

In [5]:
for cat in categories:
    cat_val = df[cat].unique()
    print('{} ({}):'.format(cat,len(cat_val)))
    print('{}\n'.format(cat_val))

code_departement (97):
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  21  22  23  24  25  26  27  28  29 120 121  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  58  59  60  61  62  63  64  65  66  69  70  71  72  73  74
  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93
  94  95 971 972 973 974  75]

code_postal (5832):
[ 1160.  1960.  1290. ... 42111. 77157. 97116.]

type_local (2):
['Maison' 'Appartement']



In [6]:
df.drop('commune', axis=1, inplace=True)
df = pd.get_dummies(df, columns=categories)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765694 entries, 0 to 765693
Columns: 5937 entries, valeur_fonciere to type_local_Maison
dtypes: float64(3), int64(3), uint8(5931)
memory usage: 4.3 GB


In [8]:
#df.to_csv(r'data/ml_data.csv', index = None, header=True)

In [9]:
y = df['valeur_fonciere']
X = df.drop(['valeur_fonciere'],axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, test_size=0.33, random_state=42, shuffle=True)


## XGBoost

In [ ]:
model = xgb.XGBRegressor()
model.fit(X_train,y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)])

In [51]:
y_train

array([355050., 128400.,  99350., ..., 336000., 180305.,  94200.])

In [ ]:
preds = xlf.predict(X_test)

print('\nMean Square error" ', mean_squared_error(y_test,preds))

## LightGBM

In [28]:
median_price = np.median(df['valeur_fonciere'].values)

In [75]:
params = {
    'boosting': 'dart',
#    'objective': 'regression',
    'metric': 'l2',
    #'max_depth': 12, 
    'learning_rate': 0.1,
    'num_iterations': 1000,
#    'verbose': 0,
    #'eval_metric'='l2',
#    'early_stopping_round': 20,
    'num_leaves':80,
    'max_depth':7,
    #'min_data_in_leaf':
}
n_estimators = 5000


In [71]:
d_train = lgb.Dataset(X_train, label=y_train)
d_test = lgb.Dataset(X_test, label=y_test)
watchlist = [d_test]

In [76]:
model = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=50)

/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[50]	valid_0's l2: 2.14997e+10
[100]	valid_0's l2: 2.0723e+10
[150]	valid_0's l2: 1.9877e+10
[200]	valid_0's l2: 1.85158e+10
[250]	valid_0's l2: 1.78384e+10
[300]	valid_0's l2: 1.7281e+10
[350]	valid_0's l2: 1.69813e+10
[400]	valid_0's l2: 1.64769e+10
[450]	valid_0's l2: 1.6087e+10
[500]	valid_0's l2: 1.58433e+10
[550]	valid_0's l2: 1.5684e+10
[600]	valid_0's l2: 1.57053e+10
[650]	valid_0's l2: 1.54749e+10
[700]	valid_0's l2: 1.53587e+10
[750]	valid_0's l2: 1.52438e+10
[800]	valid_0's l2: 1.52031e+10
[850]	valid_0's l2: 1.50287e+10
[900]	valid_0's l2: 1.49435e+10
[950]	valid_0's l2: 1.4905e+10
[1000]	valid_0's l2: 1.48276e+10


In [77]:
preds = model.predict(X_test)
#preds = np.exp(preds)

In [78]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, preds)

14827596918.597847

In [65]:
def rmsle(y_true, y_pred):
    assert len(y_true) == len(y_pred)
    return np.sqrt(np.mean(np.power(np.log1p(y_true + 1) - np.log1p(y_pred + 1), 2)))

In [46]:
preds[preds < 0] = median_price
err = rmsle(np.exp(y_test), preds)
#err_buf.append(err)
print('RMSLE = ' + str(err))

RMSLE = nan


/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/Users/odyssee/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in subtract
  This is separate from the ipykernel package so we can avoid doing imports until
